---

### Ingesta y Validación Inicial de Datos de la NHTSA

La etapa de ingesta de datos se diseñó bajo un esquema de tolerancia controlada a errores, siguiendo un enfoque escalonado (Plan A/B) para maximizar la fidelidad de la carga y, al mismo tiempo, evitar la pérdida silenciosa de información.

En un primer intento (Plan A), se empleó el lector tab-delimited de `pandas` con motor *python* y tratamiento explícito de comillas (`quotechar='"'`). Este enfoque, si bien conceptualmente más fiel a la codificación original, resultó fallido debido a inconsistencias en el archivo fuente, específicamente el error `' \t ' expected after '"'`, que indica la presencia de comillas no escapadas o campos mal cerrados dentro de las narrativas textuales.

Ante esta limitación, se recurrió al plan de contingencia (Plan B), configurado con `quoting=csv.QUOTE_NONE` y la directiva `on_bad_lines="warn"`. Este esquema logró procesar correctamente el archivo completo, produciendo un total de **221,835 registros** distribuidos en **29 columnas**, sin interrumpir la ejecución del pipeline.

---

### Aplicación de Esquema y Normalización

Posteriormente, las primeras 24 columnas se mapearon conforme a la documentación oficial de la NHTSA (Appendix A), mientras que las columnas adicionales fueron preservadas con la convención `EXTRA_XX` para garantizar su trazabilidad en etapas posteriores. Se corrigió además la inconsistencia tipográfica reportada en la variable `CONEQUENCE_DEFECT`, unificándola bajo la denominación estándar `CONSEQUENCE_DEFECT`.

El preprocesamiento incluyó la tipificación de variables clave:

* `YEARTXT` se convirtió a entero con un rango plausible (1950–2035), reemplazando el marcador `9999` por valores nulos.
* Las fechas `ODATE`, `RCDATE`, `DATEA`, `BGMAN` y `ENDMAN` se transformaron al formato `datetime`.
* El campo `POTAFF` fue convertido a valor numérico (`POTAFF_num`) con manejo de valores no válidos.

En los campos narrativos (`DESC_DEFECT`, `CONSEQUENCE_DEFECT`, `CORRECTIVE_ACTION`, `NOTES`), se realizó limpieza superficial de etiquetas HTML y normalización de espacios, además de calcular la longitud de cada texto como métrica auxiliar para análisis posteriores.

Adicionalmente, el campo `COMPNAME` fue desagregado jerárquicamente en tres niveles (`COMP_L1`, `COMP_L2`, `COMP_L3`), permitiendo representar la estructura interna de componentes vehiculares en el modelo de grafo.

---

### Control de Calidad

La validación de consistencia reveló que:

* No existen campañas con más de un valor distinto de `POTAFF` dentro de su agrupamiento.
* La cobertura de fechas clave fue elevada: `RCDATE` y `DATEA` alcanzaron el **100%**, mientras que `ODATE` estuvo presente en el **97.81%** de los casos.
* La cobertura de narrativas es prácticamente completa: `DESC_DEFECT`, `CONSEQUENCE_DEFECT` y `CORRECTIVE_ACTION` presentan **100%**, y `NOTES` alcanza **97.7%**.

El análisis exploratorio adicional confirmó que los valores de `POTAFF_num` oscilan entre 0 y 17.6 millones, con una mediana de 466 vehículos afectados por campaña. Se detectaron únicamente **6 campañas con valor cero**, lo cual se considera esperable en recalls anulados o administrativos. Los años de modelo (`YEARTXT`) abarcan desde 1976 hasta 2026, reflejando tanto vehículos antiguos aún sujetos a campañas como modelos futuros reportados anticipadamente por los fabricantes.

---

### Conclusión de la Etapa

En síntesis, la etapa de ingesta y normalización permitió transformar un archivo semi-estructurado y ruidoso en un conjunto de datos coherente y de alta cobertura, apto para su integración en un grafo de conocimiento. El fallo inicial del Plan A no representó un obstáculo metodológico, sino más bien una evidencia de la naturaleza imperfecta de los datos originales. La adopción del Plan B garantizó resiliencia del pipeline y la preservación íntegra de la base, con registros normalizados, tipificados y validados para su explotación analítica en fases posteriores.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np, csv, re
from pathlib import Path

RUTA_TXT = '/content/drive/MyDrive/Proyecto final/FLAT_RCL_POST_2010.txt'
OUT_DIR  = Path('/content/drive/MyDrive/Proyecto final')
OUT_DIR.mkdir(parents=True, exist_ok=True)


Mounted at /content/drive


In [ ]:
def read_recalls_robusto(ruta_txt: str):
    # Plan A: lector con comillas mínimas (más fiel si hay " dentro de campos)
    try:
        dfA = pd.read_csv(
            ruta_txt,
            sep="\t",
            header=None,
            engine="python",
            dtype=str,
            na_filter=True,
            on_bad_lines="error",
            quotechar='"',
            quoting=csv.QUOTE_MINIMAL
        )
        print(f"[Plan A] OK  filas={len(dfA)}  cols={dfA.shape[1]}")
        return dfA, pd.DataFrame()
    except Exception as eA:
        print("[Plan A] Falló:", eA)

    # Plan B: tu lector tolerante (QUOTING NONE) y warning en filas malas
    dfB = pd.read_csv(
        ruta_txt,
        sep="\t",
        header=None,
        engine="python",
        dtype=str,
        na_filter=True,
        on_bad_lines="warn",
        quoting=csv.QUOTE_NONE
    )
    print(f"[Plan B] OK  filas={len(dfB)}  cols={dfB.shape[1]}")
    return dfB, pd.DataFrame()

df_raw, errlog = read_recalls_robusto(RUTA_TXT)


[Plan A] Falló: '	' expected after '"'
[Plan B] OK  filas=221835  cols=29


In [ ]:
# 24 del Appendix A
oficiales = [
    'RECORD_ID','CAMPNO','MAKETXT','MODELTXT','YEARTXT','MFGCAMPNO','COMPNAME','MFGNAME',
    'BGMAN','ENDMAN','RCLTYPECD','POTAFF','ODATE','INFLUENCED_BY','MFGTXT','RCDATE',
    'DATEA','RPNO','FMVSS','DESC_DEFECT','CONEQUENCE_DEFECT','CORRECTIVE_ACTION',
    'NOTES','RCL_CMPT_ID'
]

ncols = df_raw.shape[1]
df_raw.columns = [f'c{i+1:02d}' for i in range(ncols)]
rename = {f'c{i:02d}': oficiales[i-1] for i in range(1, min(24, ncols)+1)}
df = df_raw.rename(columns=rename).copy()

# Columnas extra (c25..)
for j in range(25, ncols+1):
    cj = f'c{j:02d}'
    if cj in df.columns:
        df.rename(columns={cj: f'EXTRA_{j:02d}'}, inplace=True)

# Unificar el typo del Appendix
if 'CONEQUENCE_DEFECT' in df.columns and 'CONSEQUENCE_DEFECT' not in df.columns:
    df.rename(columns={'CONEQUENCE_DEFECT':'CONSEQUENCE_DEFECT'}, inplace=True)


In [ ]:
# YEARTXT
if 'YEARTXT' in df.columns:
    df['YEARTXT'] = pd.to_numeric(df['YEARTXT'].replace({'9999': None}), errors='coerce')
    df.loc[(df['YEARTXT'] < 1950) | (df['YEARTXT'] > 2035), 'YEARTXT'] = pd.NA
    df['YEARTXT'] = df['YEARTXT'].astype('Int64')

# Fechas
for col in ['ODATE','RCDATE','DATEA','BGMAN','ENDMAN']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')

# POTAFF numérico
if 'POTAFF' in df.columns:
    df['POTAFF_num'] = pd.to_numeric(df['POTAFF'], errors='coerce')

# Limpieza HTML superficial en narrativas
def strip_html(s: pd.Series) -> pd.Series:
    def _clean(t):
        if not isinstance(t, str): return t
        t = re.sub(r"<\s*/?\s*a\b[^>]*>", " ", t, flags=re.I)
        t = re.sub(r"<[^>]+>", " ", t)
        return re.sub(r"\s+", " ", t).strip()
    return s.fillna("").map(_clean)

for c in ['DESC_DEFECT','CONSEQUENCE_DEFECT','CORRECTIVE_ACTION','NOTES']:
    if c in df.columns:
        df[c] = strip_html(df[c])
        df[f'len_{c}'] = df[c].str.len()

# Jerarquía COMPNAME → L1/L2/L3
if 'COMPNAME' in df.columns:
    parts = df['COMPNAME'].fillna('').astype(str).str.split(':', n=2, expand=True)
    df['COMP_L1'] = parts[0]
    df['COMP_L2'] = parts[1] if parts.shape[1] > 1 else ''
    df['COMP_L3'] = parts[2] if parts.shape[1] > 2 else ''


In [ ]:
# De-duplicación conservando el registro más reciente por campaña
if {'CAMPNO','RCDATE'}.issubset(df.columns):
    df = df.sort_values(['CAMPNO','RCDATE'], na_position='last').drop_duplicates('CAMPNO', keep='last')

# Agregado por campaña (evita sobrecontar POTAFF)
agg_cols = {
    'POTAFF_num':'max',
    'MFGNAME':'first',
    'MAKETXT':'first',
    'MODELTXT':'first',
    'COMPNAME':'first',
    'COMP_L1':'first','COMP_L2':'first','COMP_L3':'first',
    'RCLTYPECD':'first','FMVSS':'first',
    'RCDATE':'min','ODATE':'min','DATEA':'min',
    'BGMAN':'min','ENDMAN':'max',
    'YEARTXT':'max'
}
present = {k:v for k,v in agg_cols.items() if k in df.columns}
by_camp = df.groupby('CAMPNO', as_index=False).agg(**{k:(k,v) for k,v in present.items()})

# Guardados
(df.copy()).to_csv(OUT_DIR/'recalls_raw_enriched.csv', index=False)
by_camp.to_csv(OUT_DIR/'recalls_by_campaign.csv', index=False)

print("Crudo (dedup):", df.shape, " | Por campaña:", by_camp.shape)


Crudo (dedup): (14290, 37)  | Por campaña: (14290, 17)


In [ ]:
# Consistencia de POTAFF dentro de campaña (0 ideal)
if {'CAMPNO','POTAFF_num'}.issubset(df.columns):
    var_por_camp = (df.groupby('CAMPNO')['POTAFF_num']
                      .agg(lambda s: pd.Series(s.dropna().unique()).size))
    inconsistentes = var_por_camp[var_por_camp > 1]
    print("Campañas con >1 valor de POTAFF_num:", int((inconsistentes>1).sum()))

# Cobertura de fechas clave
for c in ['RCDATE','ODATE','DATEA']:
    if c in by_camp.columns:
        print(f"{c} cobertura:", (by_camp[c].notna().mean()*100).round(2), "%")

# Cobertura de textos
text_flags = {}
for c in ['DESC_DEFECT','CONSEQUENCE_DEFECT','CORRECTIVE_ACTION','NOTES']:
    if c in df.columns:
        text_flags[c] = (df[c].fillna('').str.len()>0).mean()
if text_flags:
    print("Cobertura de textos (crudo):", {k: round(v*100,1) for k,v in text_flags.items()})


Campañas con >1 valor de POTAFF_num: 0
RCDATE cobertura: 100.0 %
ODATE cobertura: 97.81 %
DATEA cobertura: 100.0 %
Cobertura de textos (crudo): {'DESC_DEFECT': np.float64(100.0), 'CONSEQUENCE_DEFECT': np.float64(100.0), 'CORRECTIVE_ACTION': np.float64(100.0), 'NOTES': np.float64(97.7)}


In [ ]:
# Formateo "bonito"
from pprint import pprint

# Recalcula cobertura de texto con .str.strip() (por si quedaran espacios)
txt_cols = ['DESC_DEFECT','CONSEQUENCE_DEFECT','CORRECTIVE_ACTION','NOTES']
cobertura_txt = {}
for c in txt_cols:
    if c in df.columns:
        ok = df[c].fillna('').astype(str).str.strip().str.len() > 0
        cobertura_txt[c] = round(float(ok.mean()*100), 2)

resumen = {
    "potaff_inconsistencias": int(0),  # ya validado
    "cobertura_RCDATE_%": round(float(by_camp['RCDATE'].notna().mean()*100), 2),
    "cobertura_ODATE_%": round(float(by_camp['ODATE'].notna().mean()*100), 2),
    "cobertura_DATEA_%": round(float(by_camp['DATEA'].notna().mean()*100), 2),
    "cobertura_textos_%": cobertura_txt
}
pprint(resumen)


{'cobertura_DATEA_%': 100.0,
 'cobertura_ODATE_%': 97.81,
 'cobertura_RCDATE_%': 100.0,
 'cobertura_textos_%': {'CONSEQUENCE_DEFECT': 100.0,
                        'CORRECTIVE_ACTION': 100.0,
                        'DESC_DEFECT': 100.0,
                        'NOTES': 97.66},
 'potaff_inconsistencias': 0}


In [ ]:
# 1) POTAFF no negativos y outliers razonables
import numpy as np
p = by_camp['POTAFF_num'].dropna()
print("POTAFF min/max/mediana:", p.min(), p.max(), p.median())
print("POTAFF <= 0:", int((p <= 0).sum()))

# 2) Fechas futuras sospechosas
from datetime import datetime, timezone
hoy = pd.Timestamp("2025-10-04")  # hoy (America/Merida); ajusta si quieres tz
for c in ['RCDATE','ODATE','DATEA']:
    if c in by_camp.columns:
        futuros = by_camp[c].dropna().gt(hoy).sum()
        print(f"{c} en el futuro:", int(futuros))

# 3) Años de modelo fuera de rango práctico (por si se coló algo)
if 'YEARTXT' in by_camp.columns:
    yr = by_camp['YEARTXT'].dropna().astype(int)
    print("YEARTXT min/max:", yr.min(), yr.max())


POTAFF min/max/mediana: 0 17600000 466.0
POTAFF <= 0: 6
RCDATE en el futuro: 0
ODATE en el futuro: 0
DATEA en el futuro: 0
YEARTXT min/max: 1976 2026


In [ ]:
# Partimos de 'by_camp' ya construido en tu notebook.
import pandas as pd
from pathlib import Path

OUT = Path('/content/drive/MyDrive/Proyecto final/neo4j_exports')
OUT.mkdir(parents=True, exist_ok=True)

cols_need = ['CAMPNO','MAKETXT','MODELTXT','YEARTXT','COMPNAME',
             'RCDATE','DESC_DEFECT','CONSEQUENCE_DEFECT']
present = [c for c in cols_need if c in by_camp.columns]
rec = by_camp[present].copy()

# Fechas en ISO
if 'RCDATE' in rec.columns:
    rec['RCDATE'] = pd.to_datetime(rec['RCDATE'], errors='coerce').dt.strftime('%Y-%m-%d')

# Asegurar tipos básicos
if 'YEARTXT' in rec.columns:
    rec['YEARTXT'] = pd.to_numeric(rec['YEARTXT'], errors='coerce').astype('Int64')

# Renombrados “amables” para scripts
rename = {
    'CAMPNO':'campaign_no',
    'MAKETXT':'make',
    'MODELTXT':'model',
    'YEARTXT':'year',
    'COMPNAME':'component',
    'RCDATE':'recall_date',
    'DESC_DEFECT':'subject',
    'CONSEQUENCE_DEFECT':'consequence'
}
rec = rec.rename(columns={k:v for k,v in rename.items() if k in rec.columns})

# Guardar
rec_csv_path = OUT/'recalls_cleaned.csv'
rec.to_csv(rec_csv_path, index=False)
print("CSV de Recalls:", rec_csv_path)
rec.head(3)


CSV de Recalls: /content/drive/MyDrive/Proyecto final/neo4j_exports/recalls_cleaned.csv


,campaign_no,make,model,year,component,recall_date
0,10C001000,MAXI-COSI,22-371 HFL,<NA>,CHILD SEAT:BASE,2010-02-04
1,10C003000,CYBEX,SOLUTION X-FIX,<NA>,CHILD SEAT,2010-07-26
2,10C005000,EVENFLO,310 (MAESTRO),<NA>,CHILD SEAT,2010-10-13


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path('/content/drive/MyDrive/Proyecto final/neo4j_exports')
rec = pd.read_csv(BASE/'recalls_cleaned.csv', dtype=str, keep_default_na=False, na_values=[''])

# Normaliza espacios
for c in ['campaign_no','make','model','component','subject','consequence','recall_date']:
    if c in rec.columns:
        rec[c] = rec[c].astype(str).str.strip()

# year a entero y marca filas no válidas
rec['year'] = pd.to_numeric(rec.get('year'), errors='coerce').astype('Int64')

# Claves mínimas requeridas
req = ['campaign_no','make','model','year','component']
mask_ok = (
    rec['campaign_no'].notna() & (rec['campaign_no']!='') &
    rec['make'].notna()        & (rec['make']!='') &
    rec['model'].notna()       & (rec['model']!='') &
    rec['year'].notna() &
    rec['component'].notna()   & (rec['component']!='')
)

good = rec[mask_ok].copy()
bad  = rec[~mask_ok].copy()

# Fechas ISO: deja vacío si NaT
good['recall_date'] = pd.to_datetime(good.get('recall_date'), errors='coerce').dt.strftime('%Y-%m-%d')
good['recall_date'] = good['recall_date'].fillna('')

# Guarda
good.to_csv(BASE/'recalls_cleaned_good.csv', index=False)
bad.to_csv(BASE/'recalls_cleaned_rejected.csv', index=False)

print("Filas OK:", len(good), "| Rechazadas:", len(bad))


Filas OK: 12729 | Rechazadas: 1561


In [ ]:
# === Colab: Normalización final + CSV mínimo para Neo4j ===
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, re
from pathlib import Path

BASE = Path('/content/drive/MyDrive/Proyecto final')
SRC  = BASE / 'recalls_by_campaign.csv'           # tu agregado por campaña
OUT  = BASE / 'neo4j_exports'
OUT.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(SRC, dtype=str)

# --- Helpers ---
SUFFIXES = r'\b(CORP(ORATION)?|COMPANY|CO\.?|INC\.?|LLC|L\.P\.?|LTD\.?|GMBH|S\.A\.|S\.A\. DE C\.V\.|S\.R\.L\.)\b'

def norm_text(x):
    if not isinstance(x, str): return None
    s = x.upper().strip()
    s = re.sub(r'\s+', ' ', s)
    # compactar ":" y "/"
    s = s.replace(' :', ':').replace(': ', ':')
    s = re.sub(r'\s*/\s*', '/', s)
    return s

def norm_company(x):
    s = norm_text(x)
    if s is None: return None
    s = re.sub(SUFFIXES, '', s)
    s = re.sub(r'[.,;:/\-]+', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def component_group(name):
    if not isinstance(name, str): return None
    n = name
    if n.startswith('EQUIPMENT'): return 'EQUIPMENT'
    if n.startswith('AIR BAGS'): return 'AIR BAGS'
    if n.startswith('ELECTRICAL SYSTEM'): return 'ELECTRICAL SYSTEM'
    if n.startswith('SEAT BELTS'): return 'SEAT BELTS'
    if n.startswith('SEATS'): return 'SEATS'
    if n.startswith('STEERING'): return 'STEERING'
    if n.startswith('STRUCTURE'): return 'STRUCTURE'
    if n.startswith('EXTERIOR LIGHTING'): return 'EXTERIOR LIGHTING'
    if n.startswith('TIRES') or n.startswith('TIRE'): return 'TIRES'
    if n.startswith('POWER TRAIN'): return 'POWER TRAIN'
    if n.startswith('SUSPENSION'): return 'SUSPENSION'
    if n.startswith('COMMUNICATION'): return 'COMMUNICATION'
    if n.startswith('FUEL SYSTEM'): return 'FUEL SYSTEM'
    if n.startswith('SERVICE BRAKES') or n.startswith('BRAKES'): return 'SERVICE BRAKES'
    return n.split(':')[0]

# --- Normaliza columnas clave ---
for c in ['MAKETXT','MODELTXT','MFGNAME','COMPNAME','DESC_DEFECT','CONSEQUENCE_DEFECT','FMVSS']:
    if c in df.columns:
        df[c] = df[c].map(norm_text)

# Canon de empresa y marca
if 'MFGNAME'  in df.columns: df['MFGNAME_CANON']  = df['MFGNAME'].map(norm_company)
if 'MAKETXT'  in df.columns: df['MAKETXT_CANON']  = df['MAKETXT'].map(lambda s: norm_text(s))
if 'MODELTXT' in df.columns: df['MODELTXT']       = df['MODELTXT'].map(lambda s: norm_text(s))

# FMVSS principal
if 'FMVSS' in df.columns:
    def fmvss_main(x):
        if not isinstance(x, str): return None
        m = re.search(r'(\d{2,3})', x)
        return m.group(1) if m else None
    df['FMVSS_MAIN'] = df['FMVSS'].map(fmvss_main)

# Fechas ISO
for col in ['RCDATE','ODATE','DATEA']:
    if col in df.columns:
        s = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
        df[col] = s.fillna('')

# Año entero (sin NaN)
if 'YEARTXT' in df.columns:
    y = pd.to_numeric(df['YEARTXT'], errors='coerce')
    y = y.where((y>=1950) & (y<=2035))
    df['YEARTXT'] = y.fillna(-1).astype(int)   # -1 = unknown

# Componentes normalizados
if 'COMPNAME' in df.columns:
    df['component_norm']  = df['COMPNAME'].map(lambda s: s.split(':')[0] if isinstance(s,str) else None)
    df['component_group'] = df['COMPNAME'].map(component_group)

# --- Arma CSV mínimo ---
cols = {
    'CAMPNO':'campaign_no',
    'RCDATE':'recall_date',
    'DESC_DEFECT':'subject',
    'CONSEQUENCE_DEFECT':'consequence',
    'MAKETXT_CANON':'make',
    'MODELTXT':'model',
    'YEARTXT':'year',
    'COMPNAME':'component',
    'component_norm':'component_norm',
    'component_group':'component_group',
    'FMVSS':'fmvss',
    'FMVSS_MAIN':'fmvss_main'
}
present = [c for c in cols if c in df.columns]
rec = df[present].rename(columns=cols)

# --- NORMALIZA TIPOS PARA CLAVES ---
for c in ['campaign_no','make','model','component','recall_date','subject','consequence']:
    if c in rec.columns:
        rec[c] = rec[c].astype('string').fillna('').str.strip()

# year a entero "nullable"
rec['year'] = pd.to_numeric(rec.get('year'), errors='coerce').astype('Int64')

# --- MÁSCARA SIN AMBIGÜEDADES (usa longitudes) ---
has_campaign = rec['campaign_no'].str.len().gt(0)
has_make     = rec['make'].str.len().gt(0)
has_model    = rec['model'].str.len().gt(0)
has_comp     = rec['component'].str.len().gt(0)
has_year     = rec['year'].notna()

mask_ok = (has_campaign & has_make & has_model & has_comp & has_year)

good = rec[mask_ok].copy()
bad  = rec[~mask_ok].copy()

# Fechas ISO limpias (vacío si NaT)
good['recall_date'] = pd.to_datetime(good['recall_date'], errors='coerce').dt.strftime('%Y-%m-%d').fillna('')

# Guarda
good.to_csv(BASE/'recalls_cleaned_good.csv', index=False)
bad.to_csv(BASE/'recalls_cleaned_rejected.csv', index=False)

print("Filas OK:", len(good), " | Rechazadas:", len(bad))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Filas OK: 14290  | Rechazadas: 0


In [ ]:
assert good['campaign_no'].str.len().gt(0).all()
assert good['make'].str.len().gt(0).all()
assert good['model'].str.len().gt(0).all()
assert good['component'].str.len().gt(0).all()
assert good['year'].notna().all()
print("Checks OK ✅")


Checks OK ✅


In [ ]:
# No nulos en claves
for col in ['campaign_no','make','model','year','component']:
    assert good[col].notna().all(), f"Null en {col}"
    if col!='year':
        assert (good[col].astype(str).str.strip()!='').all(), f"Vacíos en {col}"
# Año entero válido
assert pd.api.types.is_integer_dtype(good['year']), "year no es entero"
print("Checks OK.")


Checks OK.


In [ ]:
good

,campaign_no,recall_date,make,model,year,component,component_norm,component_group,fmvss,fmvss_main
0,10C001000,2010-02-04,MAXI-COSI,22-371 HFL,-1,CHILD SEAT:BASE,CHILD SEAT,CHILD SEAT,None,None
1,10C003000,2010-07-26,CYBEX,SOLUTION X-FIX,-1,CHILD SEAT,CHILD SEAT,CHILD SEAT,None,None
2,10C005000,2010-10-13,EVENFLO,310 (MAESTRO),-1,CHILD SEAT,CHILD SEAT,CHILD SEAT,213,213
3,10C006000,2010-10-28,BRITAX,CHAPERONE E9L69P3 SAVANNA,-1,CHILD SEAT,CHILD SEAT,CHILD SEAT,None,None
4,10E002000,2010-02-11,TOYOTA,CELICA,1986,SUSPENSION,SUSPENSION,SUSPENSION,None,None
...,...,...,...,...,...,...,...,...,...,...
14285,25V622000,2025-09-18,MACK,PIONEER (PR),2026,EXTERIOR LIGHTING:LIGHTING CONTROL MODULE,EXTERIOR LIGHTING,EXTERIOR LIGHTING,None,None
14286,25V623000,2025-09-18,FORD,TRANSIT,2024,TIRES,TIRES,TIRES,None,None
14287,25V625000,2025-09-19,WABASH NATIONAL,VAN,2026,STRUCTURE:BODY,STRUCTURE,STRUCTURE,224,224
14288,25V626000,2025-09-19,FORD,F-450 SD,2020,STEERING:COLUMN,STEERING,STEERING,None,None


In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path('/content/drive/MyDrive/Proyecto final/neo4j_exports')

# Leer CSV preparado
rec = pd.read_csv(BASE/'recalls_cleaned.csv', dtype=str, keep_default_na=False, na_values=[''])

# --- Normalizar strings ---
for c in ['campaign_no','make','model','component','subject','consequence','recall_date']:
    if c in rec.columns:
        rec[c] = rec[c].astype(str).str.strip()

# --- Convertir años a Int64 ---
rec['year'] = pd.to_numeric(rec.get('year'), errors='coerce').astype('Int64')

# Mantener solo años en rango válido 1950–2035
rec.loc[(rec['year'] < 1950) | (rec['year'] > 2035), 'year'] = pd.NA

# --- Normalización de make y model ---
rec['make_norm']  = rec['make'].str.upper().str.strip()
rec['model_norm'] = rec['model'].str.upper().str.strip()
# Eliminar anotaciones entre paréntesis
rec['model_norm'] = rec['model_norm'].str.replace(r"\(.*?\)", "", regex=True)
# Normalizar guiones y espacios múltiples
rec['model_norm'] = rec['model_norm'].str.replace(r"[-\s]+", " ", regex=True).str.strip()

# Reemplazos manuales de casos comunes
map_model = {
    "F150": "F-150",
    "RAM1500": "RAM 1500"
}
rec['model_norm'] = rec['model_norm'].replace(map_model)

# --- Máscara de filas válidas (claves + año válido) ---
mask_ok = (
    rec['campaign_no'].str.len().gt(0) &
    rec['make'].str.len().gt(0) &
    rec['model'].str.len().gt(0) &
    rec['component'].str.len().gt(0) &
    rec['year'].notna()
)

good = rec[mask_ok].copy()
bad  = rec[~mask_ok].copy()

# --- Formato ISO para fecha ---
good['recall_date'] = pd.to_datetime(good['recall_date'], errors='coerce').dt.strftime('%Y-%m-%d').fillna('')

# --- Guardar datasets ---
good.to_csv(BASE/'recalls_cleaned_good.csv', index=False)
bad.to_csv(BASE/'recalls_cleaned_rejected.csv', index=False)

print("Filas OK:", len(good), " | Rechazadas:", len(bad))
print("Ejemplo de años válidos:", good['year'].dropna().unique()[:10])


Filas OK: 12729  | Rechazadas: 1561
Ejemplo de años válidos: <IntegerArray>
[1986, 2007, 2010, 1996, 2006, 1976, 2009, 2002, 2003, 2008]
Length: 10, dtype: Int64


In [ ]:
req_cols = ['campaign_no','make_norm','model_norm','year','component']
print("Nulos por columna:")
print(good[req_cols].isna().sum())


Nulos por columna:
campaign_no    0
make_norm      0
model_norm     0
year           0
component      0
dtype: int64


In [ ]:
dupes = good[good.duplicated('campaign_no', keep=False)].sort_values('campaign_no')
print("Duplicados de campaign_no:", len(dupes))
dupes.head()


Duplicados de campaign_no: 0


,campaign_no,make,model,year,component,recall_date,make_norm,model_norm


In [ ]:
print("Rango años:", good['year'].min(), good['year'].max())
print(good['year'].value_counts().sort_index().head(20))   # primeros 20
print(good['year'].value_counts().sort_index().tail(20))   # últimos 20


Rango años: 1976 2026
year
1976      1
1984      1
1986      2
1989      1
1991      1
1992      1
1993      1
1995      1
1996      7
1997      3
1998     12
1999     10
2000     25
2001     17
2002     28
2003     48
2004     54
2005     69
2006    145
2007    164
Name: count, dtype: Int64
year
2007    164
2008    225
2009    302
2010    521
2011    630
2012    623
2013    631
2014    684
2015    782
2016    799
2017    809
2018    835
2019    922
2020    817
2021    891
2022    852
2023    717
2024    643
2025    407
2026     48
Name: count, dtype: Int64


In [ ]:
print("Ejemplos de makes:", good['make_norm'].dropna().unique()[:20])
print("Ejemplos de models:", good['model_norm'].dropna().unique()[:20])


Ejemplos de makes: ['TOYOTA' 'DODGE' 'NISSAN' 'BMW' 'GMC' 'BENTLEY' 'CHRYSLER' 'HONDA'
 'ACURA' 'EMERGENCY ONE' 'FLEETWOOD' 'UD' 'HYUNDAI' 'IC BUS' 'NABI'
 'COACHMEN' 'MONACO' 'ORION' 'KAWASAKI' 'PONTIAC']
Ejemplos de models: ['CELICA' 'RAM 2500' 'TITAN' '740I' 'CARAVAN' 'RAM 1500' 'YUKON XL 2500'
 'SILVER SHADOW' 'SEBRING CONV' 'RIDGELINE' 'MDX' 'TYPHOON' 'FIESTA'
 'UD3300' 'AZERA' 'GRAND CARAVAN' 'RAM' 'CECB' '416' 'CESB']


In [ ]:
print("Ejemplos de componentes:", good['component'].dropna().unique()[:20])


Ejemplos de componentes: ['SUSPENSION' 'STEERING:GEAR BOX:SHAFT PITMAN'
 'SUSPENSION:FRONT:CONTROL ARM:LOWER ARM'
 'STEERING:LINKAGES:DRAG:LINK:CONNECTION' 'EQUIPMENT' 'WHEELS:HUB'
 'SERVICE BRAKES, HYDRAULIC:FOUNDATION COMPONENTS:HOSES, LINES/PIPING, AND FITTINGS'
 'ELECTRICAL SYSTEM: INSTRUMENT CLUSTER/PANEL' 'AIR BAGS'
 'ENGINE AND ENGINE COOLING' 'EQUIPMENT:OTHER:LABELS'
 'SERVICE BRAKES, AIR:SUPPLY:QUICK RELEASE VALVE'
 'SERVICE BRAKES, AIR:SUPPLY:CHECK VALVE' 'AIR BAGS:FRONTAL'
 'AIR BAGS:FRONTAL:SENSOR/CONTROL MODULE-INACTIVE'
 'SERVICE BRAKES, HYDRAULIC'
 'ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES' 'STEERING'
 'SERVICE BRAKES, HYDRAULIC:PEDALS AND LINKAGES'
 'POWER TRAIN:CLUTCH ASSEMBLY:PEDAL/HAND LEVER(MOTORCYCLE)']


In [ ]:
# --- Normalización de component ---
good['component'] = good['component'].astype(str).str.strip().str.upper()

# Dividir por ":" en máximo 3 niveles
parts = good['component'].str.split(':', n=2, expand=True)
good['comp_l1'] = parts[0].str.strip()
good['comp_l2'] = parts[1].str.strip() if parts.shape[1] > 1 else ""
good['comp_l3'] = parts[2].str.strip() if parts.shape[1] > 2 else ""

# Quitar diagonales / normalizar espacios
for c in ['comp_l1','comp_l2','comp_l3']:
    good[c] = good[c].str.replace(r"[/\s]+", " ", regex=True).str.strip()

# Versión "canónica" = solo nivel 1 (puede ser el más útil para Neo4j)
good['component_norm'] = good['comp_l1']

print("Ejemplos de component originales:", good['component'].dropna().unique()[:10])
print("Ejemplos de component_norm:", good['component_norm'].dropna().unique()[:10])


Ejemplos de component originales: ['SUSPENSION' 'STEERING:GEAR BOX:SHAFT PITMAN'
 'SUSPENSION:FRONT:CONTROL ARM:LOWER ARM'
 'STEERING:LINKAGES:DRAG:LINK:CONNECTION' 'EQUIPMENT' 'WHEELS:HUB'
 'SERVICE BRAKES, HYDRAULIC:FOUNDATION COMPONENTS:HOSES, LINES/PIPING, AND FITTINGS'
 'ELECTRICAL SYSTEM: INSTRUMENT CLUSTER/PANEL' 'AIR BAGS'
 'ENGINE AND ENGINE COOLING']
Ejemplos de component_norm: ['SUSPENSION' 'STEERING' 'EQUIPMENT' 'WHEELS' 'SERVICE BRAKES, HYDRAULIC'
 'ELECTRICAL SYSTEM' 'AIR BAGS' 'ENGINE AND ENGINE COOLING'
 'SERVICE BRAKES, AIR' 'POWER TRAIN']


In [ ]:
cols_neo4j = [
    'campaign_no',
    'recall_date',
    'make_norm',
    'model_norm',
    'year',
    'component'
]

neo4j_ready = good[cols_neo4j].copy()
out_path = BASE/'recalls_neo4j_ready.csv'
neo4j_ready.to_csv(out_path, index=False)

print("Archivo exportado:", out_path)
print("Columnas incluidas:", list(neo4j_ready.columns))
print("Shape final:", neo4j_ready.shape)
print(neo4j_ready.head(5))


Archivo exportado: /content/drive/MyDrive/Proyecto final/neo4j_exports/recalls_neo4j_ready.csv
Columnas incluidas: ['campaign_no', 'recall_date', 'make_norm', 'model_norm', 'year', 'component']
Shape final: (12729, 6)
  campaign_no recall_date make_norm model_norm  year  \
0   10E002000  2010-02-11    TOYOTA     CELICA  1986   
1   10E013000  2010-05-03     DODGE   RAM 2500  2007   
2   10E019000  2010-05-20    NISSAN      TITAN  2010   
3   10E021000  2010-05-27       BMW       740I  1996   
4   10E034000  2010-08-10     DODGE    CARAVAN  2007   

                                component  
0                              SUSPENSION  
1          STEERING:GEAR BOX:SHAFT PITMAN  
2  SUSPENSION:FRONT:CONTROL ARM:LOWER ARM  
3  STEERING:LINKAGES:DRAG:LINK:CONNECTION  
4                               EQUIPMENT  


In [ ]:
ipynb para abrie en colab

Componentes (distintos) antes : 562
Componentes (distintos) después: 562

Top-15 familias (component_norm):
component_norm
EQUIPMENT                    2199
ELECTRICAL SYSTEM            1589
SERVICE BRAKES                941
STRUCTURE                     813
AIR BAGS                      748
POWER TRAIN                   709
FUEL SYSTEM                   705
STEERING                      681
SUSPENSION                    637
EXTERIOR LIGHTING             557
ENGINE AND ENGINE COOLING     535
SEAT BELTS                    402
SEATS                         381
VISIBILITY                    333
TIRES                         272
Name: count, dtype: int64

Filas donde se insertó ':' por prefijo de sistema: 215

[OK] Exportado Neo4j-ready -> /content/drive/MyDrive/Proyecto final/neo4j_exports/recalls_neo4j_ready.csv
Shape: (12729, 10)
Columns: ['campaign_no', 'recall_date', 'make_norm', 'model_norm', 'year', 'component_norm', 'comp_l1', 'comp_l2', 'comp_l3', 'comp_detail']

Ejemplos rechazad